In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import time
import datetime
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from sklearn.cross_validation import StratifiedKFold
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
import jieba
#import fool
import re
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import StratifiedKFold

In [0]:
path_drive='/content/drive/My Drive/Colab Notebooks/'
final_apptype_train= pd.read_csv(path_drive+"final_apptype_train.dat",header=None,encoding='utf8',delimiter='\t')
final_apptype_train.columns=['id','conment','label']
appname_package=pd.read_csv(path_drive+"appname_package.dat",header=None,encoding='utf8',delimiter='\t')
appname_package.columns=['id','name','conment']

apptype_train= pd.read_csv(path_drive+"apptype_train.dat",header=None,encoding='utf8',delimiter=' ')
#以tab键分割，不知道为啥delimiter='\t'会报错，所以先读入再分割。
apptype_train=pd.DataFrame(apptype_train[0].apply(lambda x:x.split('\t')).tolist(),columns=['id','label','conment1'])
apptype_train['len']=apptype_train['conment1'].apply(lambda x:len(x))
#apptype_train=apptype_train[apptype_train['len']>10]

train= pd.read_csv(path_drive+"new_train.csv")
train=train[['conment1','label']]
#train=train[train.conment1!='0'].reset_index(drop=True)

test= pd.read_csv(path_drive+"new_test.csv")
test=test[['id','conment1']]

##如果conment1为0，用app包名代替
for i in tqdm(range(train.shape[0])):
    if train.loc[i,'conment1']=='0':
        train.loc[i,'conment1']=str(final_apptype_train.loc[i,'id'])
for i in tqdm(range(test.shape[0])):
    if test.loc[i,'conment1']=='0':
        test.loc[i,'conment1']=str(appname_package.loc[i,'name'])

train=pd.concat([train,apptype_train[['conment1','label']]],axis=0).reset_index(drop=True)

100%|██████████| 80000/80000 [00:47<00:00, 1680.41it/s]


In [0]:
#========================以|为分隔符，把标签分割：===============================
train['label1']=train['label'].apply(lambda x:x.split('|')[0])
train['label2']=train['label'].apply(lambda x:x.split('|')[1] if '|' in x else x) ##第二个标签有些没有，此处补0
print('训练集第一个标签分布：',train.label1.value_counts())
##去掉样本少于5个的类别,（主要考虑到后续的5折交叉验证）：
train=train[~train.label1.isin(['140805','142306','140110','142309','141703','140105'])].reset_index(drop=True)

from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
lbl.fit(train['label1'].values)
train['label1'] = lbl.transform(train['label1'].values)
num_class=train['label1'].max()+1
print('num_class',num_class)

训练集第一个标签分布： 140901    8454
140206    4004
141001    2036
142103    1575
140701    1575
140207    1488
140210    1140
140211    1129
142302    1103
140404    1047
140601     998
142104     968
140402     956
140205     747
140212     735
140304     724
140209     689
140214     646
140704     634
142308     632
140301     607
140604     589
140603     582
140202     572
140106     542
142402     532
140112     492
142701     491
140111     487
142105     487
          ... 
140303      75
140411      75
140405      74
140804      73
140410      73
140606      71
142305      70
140408      70
141301      67
142304      65
140308      63
140306      60
140703      59
142403      58
140407      52
140806      52
140412      48
141701      35
141802      30
140607      25
140808      21
140809      19
140810      19
140108      17
142309      12
142306      11
141703       9
140110       6
140805       5
140105       1
Name: label1, Length: 125, dtype: int64
num_class 119


In [0]:
pip install keras-bert

  Created wheel for keras-bert: filename=keras_bert-0.77.0-cp36-none-any.whl size=37761 sha256=d416dbe64091b66aacdfcf87b767ba994ba7ca476d351471644b157b498c3286
  Stored in directory: /root/.cache/pip/wheels/54/4f/83/b0a44df38a87c9d50cd15340d51eec24b5fb54637d86fd0352
  Created wheel for keras-transformer: filename=keras_transformer-0.30.0-cp36-none-any.whl size=13388 sha256=138c9cc97ef3ab7c58e2af79dda02c36061c351d10e61e18600badd92c879143
  Stored in directory: /root/.cache/pip/wheels/b5/06/e3/172763eea3a0b3046c91a75ec778c54e55f96ee0efdb79c044
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=21dc1fbd5da9b09e821c58082f779a6817e0b716f638107698ba0a333c844b0e
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=e424a4df8a2287099fd18f0dccc0755ede8b4d2ea1e123b254703aa51248cdde
  Stor

In [0]:
import json
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs
from keras.utils.np_utils import to_categorical
import gc
gc.collect()
maxlen = 300
config_path =path_drive+'chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path =path_drive+'chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path =path_drive+'chinese_L-12_H-768_A-12/chinese_L-12_H-768_A-12/vocab.txt'

token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

def creat_data(data):
    X1, X2, Y = [], [], []
    indexs=[]
    for i in range(len(data)):
        d =data.loc[i,'conment1']
        text = d[:maxlen]
        x1, x2 = tokenizer.encode(first=text)
        y = data.loc[i,'label1']
        if data.loc[i,'label2']!=-99:
          indexs.append([i,data.loc[i,'label2']])
        X1.append(x1)
        X2.append(x2)
        Y.append([y])
    X1 = seq_padding(X1)
    X2 = seq_padding(X2)
    Y = seq_padding(Y)
    z=to_categorical(Y)
    return [X1,X2],z

def creat_data_test(data):
    X1, X2= [], []
    for i in range(len(data)):
        d =data.loc[i,'conment1']
        text = d[:maxlen]
        x1, x2 = tokenizer.encode(first=text)
        X1.append(x1)
        X2.append(x2)
    X1 = seq_padding(X1)
    X2 = seq_padding(X2)
    return [X1,X2]

def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=10):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = range(len(self.data))
            #np.random.shuffle(idxs)
            #X1, X2, Y = [], [], []
            X1, X2= [], []
            for i in idxs:
                d = self.data.loc[i,'conment1']
                text = d[:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                #y = self.data.loc[i,'label1']
                X1.append(x1)
                X2.append(x2)
                #Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                #if i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    #Y = seq_padding(Y)
                    yield [X1, X2]#, to_categorical(Y)
                    #[X1, X2, Y] = [], [], []
                    X1, X2= [], []


Using TensorFlow backend.


In [0]:
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam


bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(num_class, activation='softmax')(x)

model = Model([x1_in, x2_in], p)
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=['accuracy']
)
model.summary()

column='conment1'
label=train.label1.values

dtest=creat_data_test(test)
N=5
stack_train = np.zeros((train.shape[0],num_class))
stack_test = np.zeros((test.shape[0],num_class))
skf=StratifiedKFold(n_splits=N,shuffle=True,random_state=42)
for i,(tr,va) in enumerate(skf.split(train[column],label)):
      
#for i, (tr, va) in enumerate(StratifiedKFold(label, n_folds=n_folds, random_state=42)):
    print('stack:%d/%d' % ((i + 1), N))
    #train_D = data_generator(traindata.loc[tr].reset_index(drop=True))
    #valid_D = data_generator(traindata.loc[va].reset_index(drop=True))

    train_D = creat_data(train.loc[tr].reset_index(drop=True))
    valid_D = creat_data(train.loc[va].reset_index(drop=True))
    if not os.path.exists(path_drive+'bert_model_%d.h5'%i):
        model.fit(train_D[0], train_D[1],
                     validation_data=(valid_D[0], valid_D[1]),
                     epochs=1,
                     batch_size=10,
                     shuffle=True,
                     )
    else:
        model.load_weights(path_drive+'bert_model_%d.h5'%i)
    #dtest=data_generator(test)
    score_te = model.predict(dtest)
    score_va = model.predict(valid_D[0])
    
    stack_train[va] += score_va
    stack_test += score_te
    
print("model acc_score:",metrics.accuracy_score(label,np.argmax(stack_train,axis=1),\
                                                normalize=True, sample_weight=None))

##获取第一第二个标签：取概率最大的前两个即可：
m=pd.DataFrame(stack_train)
first=[]
second=[]
for j,row in m.iterrows():
    zz=list(np.argsort(row))
    first.append(row.index[zz[-1]]) ##第一个标签
    second.append(row.index[zz[-2]]) ##第二个标签
m['label1']=first
m['label2']=second

#计算准确率，只要命中一个就算正确：
k=0
for i in range(len(label)):
    if label[i] in [m.loc[i,'label1'],m.loc[i,'label2']]:
        k+=1
    else:
        pass
print('线下准确率：%f'%(k/len(label)))

##准备测试集结果：
results=pd.DataFrame(stack_test)
first=[]
second=[]
for j,row in results.iterrows():
    zz=list(np.argsort(row))
    first.append(row.index[zz[-1]]) ##第一个标签
    second.append(row.index[zz[-2]]) ##第二个标签
results['label1']=first
results['label2']=second
##之前编码，最后逆编码回来：

try:
    results['label1']=results['label1'].apply(lambda x:lbl.inverse_transform(int(x)))
    results['label2']=results['label2'].apply(lambda x:lbl.inverse_transform(int(x)))
except:
    results['label1']=lbl.inverse_transform(results['label1'])
    results['label2']=lbl.inverse_transform(results['label2'])
##结合id列，保存：
pd.concat([test[['id']],results[['label1','label2']]],axis=1).to_csv(path_drive+'submit2.csv',index=None,encoding='utf8')

W0904 03:25:18.093295 140496751892352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 03:25:18.141071 140496751892352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0904 03:25:18.201928 140496751892352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0904 03:25:18.204479 140496751892352 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0904 03:25:18.215710 

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 768)          0           model_2[1][0]              

W0904 03:27:02.100062 140496751892352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 39917 samples, validate on 10039 samples
Epoch 1/1
39917/39917 [==============================] - 5819s 146ms/step - loss: 1.7125 - acc: 0.5768 - val_loss: 1.4020 - val_acc: 0.6349
stack:2/5
Train on 39942 samples, validate on 10014 samples
Epoch 1/1
 4930/39942 [==>...........................] - ETA: 1:18:51 - loss: 1.3166 - acc: 0.6485

In [0]:
##获取第一第二个标签：取概率最大的前两个即可：
m=pd.DataFrame(stack_train)
first=[]
second=[]
for j,row in m.iterrows():
    zz=list(np.argsort(row))
    first.append(row.index[zz[-1]]) ##第一个标签
    second.append(row.index[zz[-2]]) ##第二个标签
m['label1']=first
m['label2']=second

#计算准确率，只要命中一个就算正确：
k=0
for i in range(len(label)):
    if label[i] in [m.loc[i,'label1'],m.loc[i,'label2']]:
        k+=1
    else:
        pass
print('线下准确率：%f'%(k/len(label)))

##准备测试集结果：
results=pd.DataFrame(stack_test)
first=[]
second=[]
for j,row in results.iterrows():
    zz=list(np.argsort(row))
    first.append(row.index[zz[-1]]) ##第一个标签
    second.append(row.index[zz[-2]]) ##第二个标签
results['label1']=first
results['label2']=second
##之前编码，最后逆编码回来：

try:
    results['label1']=results['label1'].apply(lambda x:lbl.inverse_transform(int(x)))
    results['label2']=results['label2'].apply(lambda x:lbl.inverse_transform(int(x)))
except:
    results['label1']=lbl.inverse_transform(results['label1'])
    results['label2']=lbl.inverse_transform(results['label2'])
##结合id列，保存：
pd.concat([test[['id']],results[['label1','label2']]],axis=1).to_csv(path_drive+'submit2.csv',index=None,encoding='utf8')